In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install SpeechRecognition pydub
!apt-get install -y ffmpeg
!pip install unidecode
!pip install SpeechRecognition


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-rr613l63/unsloth_f477a86d2434472182061dbc66133d9d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-rr613l63/unsloth_f477a86d2434472182061dbc66133d9d
  Resolved https://github.com/unslothai/unsloth.git to commit e4c8ceacb3fca634f78e662873a01c37678fcb3e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.0 MB/s eta 0:00:00
   ━━

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any value till 8000 for Llama 3
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token="YOUR_HUGGINGFACE_ACCESS_TOKEN"  # Add this line if using a gated model
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [3]:
%tensorflow_version 2.x
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CorentinJ/Real-Time-Voice-Cloning.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  # install dependencies
  !cd {project_name} && pip install -q -r requirements.txt
  !pip install -q --upgrade gdown
  !apt-get install -qq libportaudio2
  !pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

  # download pretrained model
  #!cd {project_name} && wget https://github.com/blue-fish/Real-Time-Voice-Cloning/releases/download/v1.0/pretrained.zip && unzip -o pretrained.zip
  !cd {project_name} && mkdir -p saved_models/default/
  !cd {project_name}/saved_models/default/ && gdown https://drive.google.com/uc?id=1q8mEGwCkFy23KZsinbuvdKAQLqNKbYf1
  !cd {project_name}/saved_models/default/ && gdown https://drive.google.com/uc?id=1EqFMIbvxffxtjiVrtykroF6_mUh-5Z3s
  !cd {project_name}/saved_models/default/ && gdown https://drive.google.com/uc?id=1cf2NO6FtI0jDuy8AV3Xgn6leO6dHjIgu

import sys
sys.path.append(project_name)

from IPython.display import display, Audio, clear_output
from IPython.utils import io
import ipywidgets as widgets
import numpy as np
from dl_colab_notebooks.audio import record_audio, upload_audio

from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path

!ls
encoder.load_model(project_name / Path("saved_models/default/encoder.pt"))
synthesizer = Synthesizer(project_name / Path("saved_models/default/synthesizer.pt"))
vocoder.load_model(project_name / Path("saved_models/default/vocoder.pt"))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.6/676.6 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 73.8 MB/s 

/content/Real-Time-Voice-Cloning/encoder/audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Loaded encoder "encoder.pt" trained to step 1564501
Synthesizer using device: cuda
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at Real-Time-Voice-Cloning/saved_models/default/vocoder.pt


In [4]:
from datasets import load_dataset

fine_tuned_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(prompt_dict):
    instructions = prompt_dict["instruction"]
    inputs       = prompt_dict["input"]
    outputs      = prompt_dict["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = fine_tuned_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts}
pass



In [5]:
  # Load dataset from the local JSON file
dataset = load_dataset('csv', data_files='/content/prompts.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/515 [00:00<?, ? examples/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for 30% less VRAM usage
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
    output_dir="./results",
    # num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    max_steps = 60,
    learning_rate = 2E-4,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length = max_seq_length,
)

trainstats = trainer.train()

Map:   0%|          | 0/515 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 515 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss


In [8]:
def modify_response(full_text):
    # Split the full text into instruction, input, and response parts
    response_split = full_text.split("### Response:\n")
    if len(response_split) < 2:
        raise ValueError("The input text does not contain the expected format.")

    response_text = response_split[1].strip()

    # Concatenate the response text with the additional string
    modified_response  = response_text.replace("<|end_of_text|>", "").strip()
    modified_response  = modified_response.replace("'", "").strip()
    return modified_response
def generate_text(args, max_new_tokens=64, device='cuda'):
    # Enable native 2x faster inference
    FastLanguageModel.for_inference(model)  # Ensure this method is available

    # Prepare the input
    inputs = tokenizer(
        [
            fine_tuned_prompt.format(
                "You are Goti, the voice assistant for Goutam Sachdev's website, aimed at tech enthusiasts and the general public. You speak in a trendy, swaggy, yet professional tone..",  # instruction
                args,  # input
                "",  # output - leave this blank for generation!
            )
        ], return_tensors="pt").to(device)

    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)

    # Decode the output
    decoded_output = tokenizer.decode(outputs[0])

    return modify_response(decoded_output)


In [9]:
generate_text("what are you")

'I’m the voice assistant for Goutam Sachdev’s website, here to help you navigate tech topics and provide helpful resources. Goti’s got you covered!'

In [10]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
for entry in prompt:
    instruction = entry["instruction"]
    input_text = entry["input"]

    # Format the prompt
    formatted_prompt = f"{instruction} {input_text}"

    # Tokenize the input text
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    # Generate output using the model
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the generated output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"Input: {input_text}")
    print(f"Generated Output: {generated_text}")
    print("-" * 50)



NameError: name 'prompt' is not defined

In [11]:
#@title Record or Upload
#@markdown * Either record audio from microphone or upload audio from file (.mp3 or .wav)
from IPython.display import Audio
import speech_recognition as sr
import numpy as np
import wave
# Example NumPy array representing audio data
audio_data = np.random.randn(16000)  # Example NumPy array for 1 second of audio at 16kHz

# Parameters
sample_rate = 22050  # Sample rate of the audio
file_name = "temp_audio.wav"

SAMPLE_RATE = 22050
record_or_upload = "Record" #@param ["Record", "Upload (.mp3 or .wav)"]
record_seconds =   5#@param {type:"number", min:1, max:10, step:1}
embedding = None
genText= None
ss=np.empty((0,), dtype=np.float32)
def _compute_embedding(audio):
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  global embedding
  embedding = None
  embedding = encoder.embed_utterance(encoder.preprocess_wav(audio, SAMPLE_RATE))
def _record_audio():

  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  global ss
  global genText
  audio_data = np.random.randn(16000)
  ss = np.array(audio)
  audio_data = np.int16(ss / np.max(np.abs(audio_data)) * 32767)

  # Save the NumPy array as a WAV file
  with wave.open(file_name, 'w') as wf:
      wf.setnchannels(1)  # Mono audio
      wf.setsampwidth(2)  # Sample width in bytes
      wf.setframerate(sample_rate)
      wf.writeframes(audio_data.tobytes())

  # Initialize recognizer
  recognizer = sr.Recognizer()
  text=None
  # Load the audio file
  with sr.AudioFile(file_name) as source:
    audio = recognizer.record(source)

  #  Recognize the speech in the audio file
  try:
    text = recognizer.recognize_google(audio)
    genText=generate_text(args=text)
    #print("Transcription: ", text)
  except sr.UnknownValueError:
    text="Google Speech Recognition could not understand audio"
    print("Google Speech Recognition could not understand audio")
  except sr.RequestError as e:
    text="Could not request results from Google Speech Recognition service; "
    print(f"Could not request results from Google Speech Recognition service; {e}")
  return text, genText


   #_compute_embedding(audio)
def _upload_audio(b):
  clear_output()
  audio = upload_audio(sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)



In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import time
# Define the style for the terminal
terminal_style = """
<style>
    .terminal {
        background-color: black;
        color: white;
        padding: 10px;
        border-radius: 5px;
        font-family: monospace;
        white-space: pre-wrap;
        width: 600px;
        height: 300px;
        overflow: auto;
        position: relative;
    }
    .title {
        color: red;
        font-weight: bold;
        font-size: 24px;
        font-family: 'Courier New', Courier, monospace;
        text-align: center;
        margin-bottom: 10px;
    }
    .user-text {
        color: white;
    }
    .bot-text {
        color: green;
    }
</style>
"""

# Create an HTML widget for the terminal
terminal = widgets.HTML(value=terminal_style + '<div id="terminal" class="terminal"><div class="title">Goti\' Voice Assistant</div></div>')
display(terminal)

# JavaScript function to animate text
js_code = """
function animateText(text, user) {
    var terminal = document.getElementById('terminal');
    var i = 0;
    var className = user ? 'user-text' : 'bot-text';
    var div = document.createElement('div');
    div.className = className;
    terminal.appendChild(div);

    var interval = setInterval(function() {
        if (i < text.length) {
            div.innerHTML += text.charAt(i);
            i++;
        } else {
            clearInterval(interval);
        }
        terminal.scrollTop = terminal.scrollHeight;  // Auto-scroll to the bottom
    }, 50); // Adjust the interval for speed
}
"""

# Function to run JavaScript code in Colab
def run_js(text, user=True):
    display(Javascript(js_code + f"animateText('{text}', {str(user).lower()});"))

# Create a text input widget
text_input = widgets.Text(
    description='Enter text:',
    value='Welcome to the animated terminal in Google Colab!\nEnjoy coding!'
)

# Create a button to start the animation
button = widgets.Button(description='Animate Text')

# Function to handle button click
def on_button_click(b):
    text, genText=_record_audio()
    run_js(text, user=True)
    time.sleep(30)
    bot_response(genText)
    print(text, genText)


# Simulate bot response (You can call this function wherever needed)
def bot_response(text):
    run_js(text, user=False)

# Example usage
bot_response("Hello! I am Goti, your voice assistant.")
if record_or_upload == "Record":
  button = widgets.Button(description="Record Your Voice")
  button.on_click(on_button_click)
  display(button)


HTML(value='\n<style>\n    .terminal {\n        background-color: black;\n        color: white;\n        paddi…

<IPython.core.display.Javascript object>

Button(description='Record Your Voice', style=ButtonStyle())

Starting recording for 10 seconds...


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Finished recording!


<ipython-input-32-3d6ebedba0cb>:32: RuntimeWarning: divide by zero encountered in divide
  audio_data = np.int16(ss / np.max(np.abs(audio_data)) * 32767)
<ipython-input-32-3d6ebedba0cb>:32: RuntimeWarning: invalid value encountered in cast
  audio_data = np.int16(ss / np.max(np.abs(audio_data)) * 32767)


Google Speech Recognition could not understand audio


<IPython.core.display.Javascript object>

In [50]:
import ipywidgets as widgets
from IPython.display import display, Javascript
import time

# Define the style for the terminal
terminal_style = """
<style>
    .terminal {
        background-color: black;
        color: white;
        padding: 10px;
        border-radius: 5px;
        font-family: monospace;
        white-space: pre-wrap;
        width: 600px;
        height: 300px;
        overflow: auto;
        position: relative;
    }
    .title {
        color: red;
        font-weight: bold;
        font-size: 24px;
        font-family: 'Courier New', Courier, monospace;
        text-align: center;
        margin-bottom: 10px;
    }
    .user-text {
        color: white;
    }
    .bot-text {
        color: green;
    }
    .slider-text {
        color: lightgrey;
        position: absolute;
        top: 0;
        left: 100%;
        white-space: nowrap;
        animation: slide 20s linear infinite;
    }
    @keyframes slide {
        0% { left: 100%; }
        100% { left: -100%; }
    }
</style>
"""

# Create an HTML widget for the terminal
terminal = widgets.HTML(value=terminal_style + '<div id="terminal" class="terminal"><div class="title">Goti\' Voice Assistant</div></div>')
display(terminal)

# JavaScript function to animate text
js_code = """
function animateText(text, user, slider=false) {
    console.log("animateText called with text:", text, "user:", user, "slider:", slider); // Debugging log
    var terminal = document.getElementById('terminal');
    var i = 0;
    var className = slider ? 'slider-text' : user ? 'user-text' : 'bot-text';
    var div = document.createElement('div');
    div.className = className;
    terminal.appendChild(div);

    if (slider) {
        div.innerHTML = text; // Directly set the slider text
    } else {
        var interval = setInterval(function() {
            if (i < text.length) {
                div.innerHTML += text.charAt(i);
                i++;
            } else {
                clearInterval(interval);
            }
            terminal.scrollTop = terminal.scrollHeight;  // Auto-scroll to the bottom
        }, 50); // Adjust the interval for speed
    }
}
"""

# Function to run JavaScript code in Colab
def run_js(text, user=True, slider=False):
    display(Javascript(js_code + f"animateText('{text}', {str(user).lower()}, {str(slider).lower()});"))

# Create a text input widget
text_input = widgets.Text(
    description='Enter text:',
    value='Welcome to the animated terminal in Google Colab!\nEnjoy coding!'
)

# Create a button to start the animation
button = widgets.Button(description='Animate Text')

# Function to handle button click
def on_button_click(b):
    run_js("Hey I'm listening....", user=False, slider=True)
    time.sleep(2)  # Short delay before recording audio
    text, genText = _record_audio()  # Assuming you have a _record_audio function
    user_response("User: "+text)
    time.sleep(14)  # Allow time for user response animation to complete
    bot_response(genText)

# Simulate bot response (You can call this function wherever needed)
def bot_response(text):
    run_js(text, user=False)

def user_response(text):
    run_js(text, user=True)

# Example usage
bot_response("Hello! I am Goti, your voice assistant.")
record_or_upload = "Record"  # Assuming you have this variable
if record_or_upload == "Record":
    button = widgets.Button(description="Record Your Voice")
    button.on_click(on_button_click)
    display(button)

# Example for slider text effect
run_js("Welcome to Goutam Sachdev's Website.", user=False, slider=True)


HTML(value='\n<style>\n    .terminal {\n        background-color: black;\n        color: white;\n        paddi…

<IPython.core.display.Javascript object>

Button(description='Record Your Voice', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

Starting recording for 5 seconds...


<IPython.core.display.Javascript object>

Finished recording!
Google Speech Recognition could not understand audio


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>